ИМПОРТИРОВАНИЕ НЕОБХОДИМЫХ ДЛЯ РАБОТЫ БИБЛИОТЕК

In [18]:
import glob
import cv2
import re
import numpy as np
from PIL import Image
from pillow_heif import register_heif_opener
import time
import datetime

register_heif_opener()

path = r'images/heic'
extension = path.split('/')[-1]
filenames = glob.glob(f'{path}/*.{extension}')
images_dict = {}

ПРОЦЕДУРА ПОИСКА КОНТУРА ПО ЗАДАННЫМ ПАРАМЕТРАМ:
 - ВХОД: source image, kernel size, thresh value, erosian value, iterations, channel number, method
 - ВЫХОД: thresh, cnt, contour

In [19]:
def finder(res_image, kernel, thr, er, iter, channel, method):
    res_image = cv2.blur(res_image, (kernel, kernel))
    _, thresh = cv2.threshold(res_image[:,:,channel], thr, 255, method)
    thresh = cv2.erode(thresh, np.ones((er, er), np.uint8), iterations=iter)
    contour, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    cnt = sorted(contour, key=cv2.contourArea)[-1]

    return thresh, cnt, contour

ЗАГРУЗКА ИЗОБРАЖЕНИЙ, ИЗМЕНЕНИЕ РАЗРЕШЕНИЯ ЗАГРУЖЕННЫХ ИЗОБРАЖЕНИЙ

In [20]:
start = time.time()

for filename in filenames:

    img = np.asanyarray(Image.open(filename))
    assert img is not None, 'file could not be read'

    h, w = img.shape[:2]
    k = h / w
    s = int(h / 8)    
    image = cv2.resize(img.copy(), (int(s / k), s), interpolation=cv2.INTER_CUBIC)

    images_dict[filename] = image

interim = time.time()

ПОИСК КОНТУРОВ С ОТРИСОВКОЙ НА ИСХОДНОМ ИЗОБРАЖЕНИИ

In [ ]:
for key, value in images_dict.items():
    thresh, cnt, c = finder(value, 5, 121.5, 1, 3, 1, cv2.THRESH_BINARY_INV)
    mask = np.zeros(value.shape[:2], np.uint8)

    M = cv2.drawContours(mask.copy(), [cnt], -1, (255,255,255), -1)

    masked = cv2.subtract(cv2.bitwise_and(value, value, mask = M), 35)

    thresh_1, cnt_1, cnts = finder(masked, 3, 0.8, 3, 3, 1, cv2.THRESH_BINARY)

    cv2.drawContours(value, [cnt], -1, (0, 255, 0), 1)
    cv2.drawContours(value, [cnt_1], -1, (0, 0, 255), 1)

    c = cv2.contourArea(cnt)
    c_1 = cv2.contourArea(cnt_1)
    bark = np.round(((c - c_1) / c) * 100, 2)

    cv2.putText(value, f'Bark area = {bark}%', (20, value.shape[:2][0] - 40), cv2.FONT_HERSHEY_SIMPLEX, (s / 600), (255, 0, 0), int(round(s / 300)))
    file_number = re.findall(r'\d+', key)[0]
    cv2.imwrite(f'images/experiment/IMG_{file_number}.png', value[:,:,::-1])
end = time.time()

ВЫВОД РЕЗУЛЬТАТОВ РАБОТЫ ПРОГРАММЫ В КОНСОЛЬ

In [39]:
t = str(datetime.timedelta(seconds = (end - start).__round__(0)))
t_1 = str(datetime.timedelta(seconds = (interim - start).__round__(0)))
t_2 = str(datetime.timedelta(seconds = (end - interim).__round__(0)))
print(f'Total program execution time:\t\t\t{t}\nTime for reading and preprocessing images:\t{t_1}\nTime for processing images:\t\t\t{t_2}')
print(f'Number of images processed:\t\t\t{len(filenames)}')
print('The resulting images are saved to the folder \'Experiment\'')

Total program execution time:			0:00:22
Time for reading and preprocessing images:	0:00:21
Time for processing images:			0:00:01
Number of images processed:			45
The resulting images are saved to the folder 'Experiment'
